### Imports

In [ ]:
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import pickle

from prerequisites import *

PATH definition

In [ ]:
DUMPS_PATH = Path(Path.cwd(), "04_attack_recognition_trained")

DS_SCHEMA = label_casting("00_type_cast_data/type_list.txt")
COLUMN_LIST = column_list_reader("00_type_cast_data/column_list.txt")

### Reading the importance data

In [ ]:
attacks_importances = []

for dump in DUMPS_PATH.iterdir():
    # filter attack that are not DDoSes
    # if dump.stem.find("DDoS-ICMP") == -1:
    #     continue
    
    attack_trained_pickled = open(dump, "rb")
    forest_classifier = pickle.load(attack_trained_pickled)
    attack_trained_pickled.close()
    
    importances = forest_classifier.feature_importances_
    
    feat_list = []
    for col, imports in zip(COLUMN_LIST, importances):
        feat_list.append((col, imports))
        
    dtypes = [("feature", "U50"), ("importance", np.float64)]
    
    arr = np.array(feat_list, dtype=dtypes)
    # arr[::-1].sort(order="importance",)
    
    attacks_importances.append((dump.stem[5:], np.sort(arr, order="importance",)[::-1]))    

### Plotting the results

In [ ]:
PLOT_PATH = Path(Path.cwd(), "FINAL_feats_plots")
TOP_NUM = 30

for att in attacks_importances:
    arr_top = att[1][:TOP_NUM]
    plt.rcParams["figure.figsize"] = (10, 5)
    plt.bar(arr_top["feature"], [el*100 for el in arr_top["importance"]], zorder=3, color="#4AA160")
    plt.title("{} - Feature Importance (top {})".format(att[0][:-9], TOP_NUM))
    plt.xticks(rotation=90)
    plt.ylabel("Importance (%)")
    plt.tight_layout()
    plt.grid(zorder=0)
    plt.savefig(Path(PLOT_PATH, "{}.pdf".format(att[0][:-9])), bbox_inches='tight')
    plt.show()